# 교차 검증

## 과적합 (Overfitting)
- model이 학습 data에만 과도하게 최적화
- 실제 예측을 수행하는 경우 예측 성능이 떨어짐

## 교차검증
- 과적합 문제를 해결하기 위함
- 데이터의 편중을 막기 위해 여러set로 구성된 train dataset과 검증 dataset에서 학습과 평가 수행
- 각 세트에서 수행한 평가 결과에 따라 하이퍼 파라미터 튜닝등의 모델 최적화 가능

### 1) K 폴드 교차 검증
- k개의 데이터 폴드 세트를 만들어서 각 폴드 세트에 학습과 검증 평가를 반복적 수행

#### 순서
1. dataset을 K등분 함
2. 첫번째 반복에서 (k-1)개를 학습 dataset, 1개를 검증 dataset으로 설정
3. 학습 dataset에서 학습 수행 검증 dataset에서 평가 수행
4. 검증 dataset을 바꿔가면서 총 k번 반복 수행
5. K개의 예측 평가 구하면 평균해서 K 폴드 평가 결과로 반여

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

C:\Users\admin\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
iris=load_iris()

In [3]:
features=iris.data
label=iris.target
clf=DecisionTreeClassifier(random_state=156)

In [4]:
kfold=KFold(n_splits=5) # 5개의 폴드 세트로 분리하는 KFold 객체
acc=[] # 폴드 세트 별 정확도를 담을 리스트

In [5]:
# iris에는 총 150개의 data들이 있음
features.shape[0]

150

### KFold 객체에 split() 호출해 전체 data를 5개로 분리


In [6]:
n_iter=0

In [9]:
# k Fold 객체의 split()호출하면 폴드 별 학습용, 검증용 테스트의 row index를 array로 반환
for train_index, test_idx in kfold.split(features):
    # k Fold.split() 로 반환 된 index 이용해 학습용, 검증용 test data 추출
    X_train, X_test=features[train_index], features[test_idx]
    y_train, y_test=label[train_index],label[test_idx]

    # 학습 및 예측 
    clf.fit(X_train, y_train)
    pred=clf.predict(X_test)
    n_iter+=1

    # 반복 시마다 정확도 측정
    accuracy=np.round(accuracy_score(y_test,pred),4)
    train_size=X_train.shape[0]
    test_size=X_test.shape[0]
    print(n_iter, accuracy,train_size,test_size)
    acc.append(accuracy)

3 1.0 120 30
4 0.9667 120 30
5 0.8667 120 30
6 0.9333 120 30
7 0.7333 120 30


In [10]:
# 각 횟수 별 정확도 평균 내서 평균 정확도 계산 
np.mean(acc)

0.9

### Result
- 5번의 교차 검증 결과 평균 검증 정확도는 0.9

## 2) Stratified K 폴드 교차 검증
- 불균형한 분포도를 가진 레이블( 결정 class ) 데이터 집합을 위한 k 폴드 방식
- label data 집합이 원본 데이터 집합의 label 분포를 train/test set에 제대로 분배하지 못하는 경우 문제 해결
- 원본 data의 label분포를 먼저 고려한 뒤 분포와 동일하게 dataset 분배
- 특정 레이블 값이 특이하게 많거나 매우 적어서 값의 분포가 한쪽으로 치우치는 경우 사용
    - Ex) 대출 사기 데이터 예측하는 경우

In [15]:
from sklearn.model_selection import StratifiedKFold
import pandas as pd

C:\Users\admin\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\admin\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [16]:
irisdf=pd.DataFrame(data=iris,columns=iris.feature_names)
irisdf['label']=iris.target

In [17]:
skf=StratifiedKFold(n_splits=3)
n_iter=0

In [19]:
for train_index, test_idx in skf.split(irisdf,irisdf['label']):
    n_iter+=1
    label_train=irisdf['label'].iloc[train_index]
    label_test=irisdf['label'].iloc[test_idx]
    print("교차검증", n_iter)
    print("학습 레이블 데이터 분포\n", label_train.value_counts())
    print("test 레이블 데이터 분포\n", label_test.value_counts())

교차검증 4
학습 레이블 데이터 분포
 2    33
1    33
0    33
Name: label, dtype: int64
test 레이블 데이터 분포
 2    17
1    17
0    17
Name: label, dtype: int64
교차검증 5
학습 레이블 데이터 분포
 2    33
1    33
0    33
Name: label, dtype: int64
test 레이블 데이터 분포
 2    17
1    17
0    17
Name: label, dtype: int64
교차검증 6
학습 레이블 데이터 분포
 2    34
1    34
0    34
Name: label, dtype: int64
test 레이블 데이터 분포
 2    16
1    16
0    16
Name: label, dtype: int64


### Result
- 학습 label & 검증 label data값의 분포도가 동일하게 할당

In [20]:
dt_clf=DecisionTreeClassifier(random_state=156)

In [21]:
skfold=StratifiedKFold(n_splits=3)
n_iter=0
cv_accuracy=[]

In [22]:
for train_idx, test_idx in skfold.split(features,label):
    X_train, X_test=features[train_idx],features[test_idx]
    y_train, y_test= label[train_idx], label[test_idx]
    # 학습 & 예측
    dt_clf.fit(X_train, y_train)
    pred=dt_clf.predict(X_test)

    # 반복 시마다 정확도 측정
    n_iter+=1
    accuracy=np.round(accuracy_score(y_test,pred),4)
    train_size=X_train.shape[0]
    test_size=X_test.shape[0]
    cv_accuracy.append(accuracy)

In [23]:
print('교차 검증 별 정확도', np.round(cv_accuracy,4))
print(' 평균 검증 정확도 ', np.mean(cv_accuracy))

교차 검증 별 정확도 [0.9804 0.9216 0.9792]
 평균 검증 정확도  0.9604


### Result
- 평균 검증 정확도가 96.04% 로 측정
- 왜곡된 label dataset에서는 반드시 Stratified K 폴드 이용해 교차 검증

## Cross_val_score()
- 교차 검증 편리하게 수행 가능
- Stratified K 폴드 방식으로 label값의 분포에 따라 Dataset 분할
- 회귀의 경우에는 k 폴드 방식으로 수행
- 내부에서 estimator를 학습(fit), 예측(predict), 평가(evaluation) 시켜줌

<br> </br>

- Options
    - estimator : Classifier & Regressor
    - X : feature Dataset
    - Y : Label Dataset
    - scoring : 예측 성능 평가 지표
    - cv : 교차 검증 폴드 수 의미

    

In [32]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score,cross_validate
from sklearn.datasets import load_iris

In [27]:
iris=load_iris()

In [28]:
dt_clf=DecisionTreeClassifier(random_state=156)

In [29]:
data=iris.data
label=iris.target

In [31]:
# 성능 지표는 정확도 (accuracy), 교차 검증 set는 3개
scores=cross_val_score(dt_clf,data,label,scoring='accuracy',cv=3)
print("교차 검증별 정확도", np.round(scores,4))
print("평균 검증 정확도", np.round(np.mean(scores),4))

교차 검증별 정확도 [0.9804 0.9216 0.9792]
평균 검증 정확도 0.9604


## GridSearchCV 
- 교차 검증과 최적 하이퍼 파라미터 튜닝을 한번에 해줌
- Hyper Parameter를 순차적으로 입력하면서 최적의 Parameter 도출 방안을 제공
- train data를 cv만큼 폴딩 셑트로 분할 & param_grid 에 있는 파라미터를 순차적 변경하면서 학습/평가 수행 
- fit()수행 이후 최고 성능을 나타낸 hyper Parameter 값과 그때의 평가 결과가 best_paeams_ , best_score_ 속성에 저장
- 최적 성능 나타내는 hyper Parameter로 Estimator학습해 best_estimator_로 저장
   
### Hyper Parameter
- 이 값을 조정해 알고리즘의 예측 성능 개선 가능


#### Ex) Decision Tree Algorithm의 hyper Parameter를 순차적으로 변경하면서 최고 성능 가지는 parameter조합 찾기
- Decision Tree의 중요 hyper parameter : max_depth & min_sample_split


In [33]:
grid_parameter={'max_depth':[1,2,3],
            'min_samples_split':[2,3]
}

In [43]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV,train_test_split


In [44]:
iris=load_iris()

In [45]:
x_train,x_test,y_train,y_test=train_test_split(iris.data,iris.target, test_size=0.2, random_state=121)

In [46]:
dtree=DecisionTreeClassifier()

In [55]:
# Dictionary 형태로 Parameter 설정
parameter={'max_depth':[1,2,3],
            'min_samples_split':[2,3]}

In [50]:
import pandas as pd
# Param_grid의 하이퍼 파라미터를 3개의 train,test set fold로 나누어 테스트
# refit =True가 default. ( 가장 좋은 parameter 설정으로 재학습 시킴)
grid_dtree=GridSearchCV(dtree,param_grid=parameter,cv=3,refit=True)

In [51]:
# Param_grid의 하이퍼 파라미터를 순차적으로 학습 / 평가
grid_dtree.fit(x_train,y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=None,
                                              splitter='best'),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': [1, 2, 3], 'min_samples_split': [2, 3]},
             pre_dispatch='2*n_jobs', refit=True, return_

In [53]:
scores_df=pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params','mean_test_score','rank_test_score','split0_test_score','split1_test_score','split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


### Result
- params : 수행시에 적용 된 hyper parameter 값
- rank_test_score 가 1인 것이 예측 성능이 1위
- mean_test_score : cv의 폴딩 테스트 셋에 대해 총 수행한 평가 평균값

<br> </br>

max_depth=3 & min_samples_split=2 일때 rank_test_score=1이므로 예측 성능이 1위  
이때의 mean_test_score=0.975로 제일 높은 것을 알 수 있음

In [56]:
print("최적의 parameter", grid_dtree.best_params_)
print('최고 정확도', grid_dtree.best_score_)

최적의 parameter {'max_depth': 3, 'min_samples_split': 2}
최고 정확도 0.975


In [60]:
# refit으로 이미 학습된 estimator반환
estimator=grid_dtree.best_estimator_

In [58]:
pred=estimator.predict(x_test)
print("test dataset 정확도", accuracy_score(y_test,pred))

test dataset 정확도 0.9666666666666667
